<a href="https://colab.research.google.com/github/Jaime44/KUNVENO/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

In [1]:
import sys
import os


# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/PRÁCITCAS EXT/KUNVENO/KUNVENO_DIGITAL/'


pat_dir_checkpoints = path_absolute+'checkpoints'

El código se está ejecutando en Google Colab.
Mounted at /content/drive
Contenido de la carpeta en Google Drive:
['Git_commnads.txt', '.git', 'README.md', 'checkpoints', 'icefall', '.gitignore', '.gitmodules', 'KUNVENO_DIGITAL', 'icefall_yes_no_dataset_recipe_with_CPU.ipynb']


In [2]:
!ls -la

total 34
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   399 Oct 11 18:53 .gitignore
-rw------- 1 root root    79 Oct 11 18:53 .gitmodules
drwx------ 2 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 13587 Oct 12 16:26 icefall_yes_no_dataset_recipe_with_CPU.ipynb
drwx------ 2 root root  4096 Oct 12 15:51 KUNVENO_DIGITAL
-rw------- 1 root root    17 Oct 11 16:57 README.md


In [3]:
!pwd

/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL


### Install PyTorch and torchaudio

In [4]:
import torch
print(torch.__version__)

2.0.1+cu118


Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [5]:
! pip install torchaudio==2.0.2

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [6]:
! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 MB 8.9 MB/s eta 0:00:00


Check that k2 was installed successfully:

In [7]:
! python3 -m k2.version


k2 version: 1.24.3
Build type: Release
Git SHA1: e400fa3b456faf8afe0ee5bfe572946b4921a3db
Git date: Sat Jul 15 04:21:50 2023
Cuda used to build k2: 11.8
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.26.4
GCC version: 9.3.1
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -u

### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [8]:
! pip install lhotse
! pip install git+https://github.com/lhotse-speech/lhotse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.9/731.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 11.1 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=0b3b922ddd828cbdef6d76055e62a577f3d27ff1656c995330e7de8595dc62f3
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-0qikv_dx
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-0qikv_dx
  Resolved https://github.com/lhotse-speech/lhotse to commit afc8eff1d1eaeeae42e0a628e1981bb231527340
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lhotse: filename=lhotse-1.17.0.dev0+git.afc8eff.clean-py3-none-any.whl size=732280 sha256=75effc2e8b61fca3932fa644f060f827e3fbeba8ac68ab57e165892522752158
  Stored in directory: /tmp/pip-ephem-wheel-cache-pasw8h8a/wheels/df/b0/ff/cce0f16868fcdbee2088f3acf9f249dc90117d5f5dd9b6f69d
Successfully built lhotse
  Attempting uninstall: lhotse
    Found existing installation: lhotse 1.17.0
    Uninstalling lhotse-1.17.0:
      Successfully uninstalled lhotse-1.17.0


### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [9]:
! pwd

/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL


In [10]:
# ! git clone https://github.com/k2-fsa/icefall
# !git clone --recursive https://github.com/Jaime44/KUNVENO_DIGITAL.git


Now install dependencies of `icefall`:

In [11]:
! cd icefall && \
  pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you ha

## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [12]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0


In [13]:
# !export PYTHONPATH=icefall:$PYTHONPATH

In [14]:

!ls -la icefall/egs/yesno/ASR


total 16
drwx------ 2 root root 4096 Oct 11 18:53 local
-rw------- 1 root root 2163 Oct 11 18:53 prepare.sh
-rw------- 1 root root  349 Oct 11 18:53 README.md
-rw------- 1 root root   24 Oct 11 18:53 shared
drwx------ 2 root root 4096 Oct 11 18:53 tdnn
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [15]:
# !chmod 755 icefall/egs/yesno/ASR/prepare.sh

In [16]:

!ls -la icefall/egs/yesno/ASR

total 16
drwx------ 2 root root 4096 Oct 11 18:53 local
-rwx------ 1 root root 2163 Oct 11 18:53 prepare.sh
-rw------- 1 root root  349 Oct 11 18:53 README.md
-rw------- 1 root root   24 Oct 11 18:53 shared
drwx------ 2 root root 4096 Oct 11 18:53 tdnn
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [17]:
! export PYTHONPATH=./icefall/:$PYTHONPATH && \
  cd icefall/egs/yesno/ASR && \
  rm -rf data && \
  ./prepare.sh

/usr/bin/env: ‘bash\r’: No such file or directory


In [18]:
pwd

'/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL'

In [19]:
!pwd


/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL


## Training

In [20]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/train.py

/bin/bash: line 1: export: `portátil/KUNVENO/KUNVENO_DIGITAL/icefall:/env/python': not a valid identifier


## Decoding

In [21]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/decode.py

/bin/bash: line 1: export: `portátil/KUNVENO/KUNVENO_DIGITAL/icefall:/env/python': not a valid identifier


### Show the decoding result

In [22]:
! cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cat tdnn/exp/recogs-test_set.txt

/bin/bash: line 1: cd: too many arguments


### Show the detailed WER

In [23]:
! cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cat tdnn/exp/errs-test_set.txt

/bin/bash: line 1: cd: too many arguments


# Pre-trained model

### Download the pre-trained model

In [24]:
# ! cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   mkdir tmp && \
#   cd tmp && \
#   git lfs install && \
#   git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn

In [25]:
# ! sudo apt-get install git-lfs

In [26]:
# ! cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   mkdir -p tmp && \
#   tree tmp

In [27]:
# ! sudo apt-get install tree

In [28]:
# ! cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   mkdir -p tmp && \
#   tree tmp

In [29]:
# ! sudo apt-get install sox

In [30]:
# ! cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [31]:
# ! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

## Inference with a pre-trained model


### View help information

In [32]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py --help

### Decode a single sound file

In [33]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

### Decode multiple sound files

In [34]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav




**-----------------------------------------------------------------------------------------------------------------------------------------------------------**